## Imports

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from model import Neuro_gambit

## ELO init

In [ ]:
elo = 2000

## Device init

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # using cuda
cpu = torch.device('cpu') # using cpu

## Loading Tensors

In [ ]:
X = torch.load('./data/X_tensor_'+str(elo)+'.pt').to(device)
Y = torch.load('./data/Y_tensor_'+str(elo)+'.pt').to(device)
print(X.shape)
print(Y.shape)

# seperating the Y
Y1 = Y[:, :8]
Y2 = Y[:, 8:16]
Y3 = Y[:, 16:24]
Y4 = Y[:, 24:32]
Y5 = Y[:, 32:]

Y = [Y1,Y2,Y3,Y4,Y5]

## Model class init

In [ ]:
model = Neuro_gambit().to(device)

# epochs, loss, and optim
learning_rate = 0.01
n_epochs = 1000000

# loss and optimizer functions from pytorch
criterion = nn.MSELoss() # MSE function
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate) # stochastic gradient descent function

## Training

In [ ]:
for epoch in range(n_epochs):
    # forward
    y_preds = model(X) # will output a tuple of 5 tensors

    total_loss = 0
    for i in range(len(y_preds)): # calculating the loss per tensor
        y_pred = y_preds[i]
        total_loss += criterion(y_pred, Y[i])

    # backward
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss.item():.4f}', end='\r')

## Saving the model

In [ ]:
# Save the model
torch.save(model.state_dict(), './models/neuro_gambit_'+str(elo)+'.pt')

## Loading saved model

In [ ]:
model.load_state_dict(torch.load('./models/neuro_gambit_'+str(elo)+'.pt')) # it takes the loaded dictionary, not the path file itself
model.eval()

## Sample test

In [ ]:
import torch.nn.functional as F

import chess
from chess_transformation import linear_to_matrix, matrix_to_board, full_move_to_algebraic

def get_all_possible_moves(board : chess.Board, player : str):
    possible_moves = []
    player_col = chess.WHITE if player == 'white' else chess.BLACK
    for move in board.legal_moves:
        if board.turn == player_col:
            uci_move = move.uci()
            possible_moves.append(uci_move)

    return possible_moves

def get_move_probability(move : str, tensor_tuple : tuple[torch.Tensor]):
    pos_rank_labels = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    pos_file_labels = {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7}

    # restruct tensors to list matrix
    tensor_matrix = []
    for tensor in tensor_tuple[:4]:
        tensor_list = F.softmax(tensor, dim=0).to(cpu).tolist()
        tensor_matrix.append(tensor_list)

    # get indices
    o_r_i = pos_rank_labels[move[0]]
    o_f_i = pos_file_labels[move[1]]
    d_r_i = pos_rank_labels[move[2]]
    d_f_i = pos_file_labels[move[3]]

    # get probabilities
    p_o_r = tensor_matrix[0][o_r_i]
    p_o_f = tensor_matrix[1][o_f_i]
    p_d_r = tensor_matrix[2][d_r_i]
    p_d_f = tensor_matrix[3][d_f_i]

    return p_o_r*p_o_f*p_d_r*p_d_f

def get_best_move(model : Neuro_gambit, board : list[str] | chess.Board, player : str):
    if type(board) != chess.Board:
        board : chess.Board = matrix_to_board(linear_to_matrix(board), player)
    possible_moves = get_all_possible_moves(board, player)
    with torch.no_grad():
        output_tensor = model.forward_chess_board_input(board, player)
    output_tensor_formatted = [t.view(-1,) for t in output_tensor]
    possible_moves_probabilities = {move : get_move_probability(move, output_tensor_formatted) for move in possible_moves}
    max_move_prob = {'move' : '', 'prob' : 0}
    for move in possible_moves_probabilities.keys():
        prob = possible_moves_probabilities[move]
        if prob > max_move_prob['prob']:
            max_move_prob['move'] = move
            max_move_prob['prob'] = prob

    if len(max_move_prob['move']) == 5: # there is a promotion
        promotion_prediction = output_tensor_formatted[4].softmax(0).tolist()
        index = promotion_prediction.index(max(promotion_prediction))
        promotion_labels = {0 : 'q', 1 : 'r', 2 : 'b', 3 : 'n'}
        max_move_prob['move'] = max_move_prob['move'][:4]+promotion_labels[index]

    return max_move_prob

In [ ]:
# Playing a game: poor performance. Model 2500 elo
b = chess.Board()
ai_col = 'black'

b.push_san('e4')
b.push_uci('c7c5')

b.push_san('Bc4')
b.push_uci('g8f6')

b.push_san('d3')
b.push_uci('d8c7')

b.push_san('Nf3')
b.push_uci('e8d8')

b.push_san('Ng5')
b.push_uci('b7b5')

b.push_san('Nxf7+')
b.push_uci('d8e8')

b.push_san('Nxh8')
b.push_uci('e8d8')

b.push_san('Bxb5')
b.push_uci('c8b7')

b.push_san('Nf7+')
b.push_uci('d8c8')

b.push_san('O-O')
b.push_uci('b7d5')

b.push_san('exd5')
b.push_uci('h7h5')

b.push_san('Nc3')
b.push_uci('d7d6')

b.push_san('Bc6')
b.push_uci('c7d8')

b.push_san('Bxa8')
b.push_uci('c5c4')

b.push_san('dxc4')

print(b)
best = get_best_move(model, b, ai_col)
print(best)

In [ ]:
# Playing a game : huge mistake made. Model 2500 elo
b = chess.Board()
ai_col = 'white'

b.push_san('e4')
b.push_san('d5')

b.push_san('exd5')
b.push_san('Qxd5')

b.push_san('Nc3')
b.push_san('Qe6+')

b.push_san('Ne4')
b.push_san('Qxe4+')

b.push_san('Qe2')
b.push_san('Qxc2')

b.push_san('Qg4')


print(b)
best = get_best_move(model, b, ai_col)
print(best)